## Exercise - Gandhi Clothing Company

Manufacture shirts, shorts and pants.

Shirts - $200 per week machinery, 3 hours labor, 4 sq. yards of cloth

Shorts - $150 per week machinery, 2 hours labor, 3 sq. yards of cloth

Pants - $100 per week machinery, 6 hours labor 4 sq. yards of cloth

The profit equation is:

<center> $ \max Profit = \max (\sum_iPrice_{i} \times x_{i} - \sum Variable Costs_i  \times x_i -\sum Fixed Costs_i \times b_i), \space\space\space\space b_{i}$ is binary variable

$s.t.$

$\sum Labor_{i} \times x_{i} \le 150$

$\sum Cloth_{i} \times x_{i} \le 160$

$ x_i \le M_i \times b_i$

where $M_i$ is the maximum amount of clothing item $i$ that can  be made within the total cloth constraint (160 yards).

$x_i \ge 0 \space\space\space \forall i$

In [ ]:
! pip install -q pyomo
! pip install cplex -q
!apt-get install -y -qq coinor-cbc

In [ ]:
import pandas as pd


# Load the Excel file
xls = pd.ExcelFile('/content/drive/MyDrive/Colab Notebooks/Optimization with Python/Linear Programming/Gandhi_Clothing.xlsx')

# Read each sheet
df = xls.parse('grid')

In [ ]:
# Extract sets

param_dict = {}

for i in range(len(df.columns)):
  param_dict[df.columns[i]] = df.iloc[:,i].tolist()

param_dict

{'Item': [0, 1, 2],
 'Price': [12, 8, 15],
 'cost': [6, 4, 8],
 'labor hours': [3, 2, 6],
 'cloth': [4, 3, 4],
 'Machine': [200, 150, 100],
 'Maximum': [40, 53, 40]}

Set up parameters for each clothing type

In [ ]:
from pyomo.environ import *

model = ConcreteModel()

# Define set of clothing types
model.item = Set(initialize=param_dict['Item'])  #  {0: 'Shirts , 1: 'Shorts', 2: 'Pants'}

model.price = Param(model.item, initialize=dict(zip(param_dict['Item'], param_dict['Price'])))
model.labor = Param(model.item, initialize=dict(zip(param_dict['Item'], param_dict['labor hours']))) # 'labor hours'
model.cloth = Param(model.item, initialize=dict(zip(param_dict['Item'], param_dict['cloth']))) # 'cloth'
model.variable_cost = Param(model.item, initialize=dict(zip(param_dict['Item'], param_dict['cost'])))
model.fixed_cost = Param(model.item, initialize=dict(zip(param_dict['Item'], param_dict['Machine'])))
model.maximum = Param(model.item, initialize=dict(zip(param_dict['Item'], param_dict['Maximum'])))

price = model.price
labor = model.labor
cloth = model.cloth
variable_cost = model.variable_cost
fixed_cost = model.fixed_cost
maximum = model.maximum

model.x = Var(model.item, domain=NonNegativeIntegers, initialize = dict(zip(param_dict['Item'], param_dict['Maximum'])))
model.b = Var(model.item, domain=Binary) # Define binary variable b

In [ ]:
def net_profit(model):
    return sum(model.price[i] * model.x[i] for i in model.item) - sum(model.variable_cost[i] * model.x[i] for i in model.item) - sum(model.fixed_cost[i] * model.b[i] for i in model.item)

model.obj = Objective(rule=net_profit, sense=maximize)

Define constraints

In [ ]:
def labor_constraint(model):
    return sum(model.labor[i] * model.x[i] for i in model.item) <= 150

model.labor_constraint = Constraint(rule=labor_constraint)

In [ ]:
def cloth_constraint(model):
    return sum(model.cloth[i] * model.x[i] for i in model.item) <= 160

model.cloth_constraint = Constraint(rule=cloth_constraint)

In [ ]:
def maximum_constraint(model, i):
    return model.x[i] <= model.maximum[i] * model.b[i]

model.maximum_constraint = Constraint(model.item, rule=maximum_constraint)

In [ ]:
solver = SolverFactory('cbc')  # cbc solver
results = solver.solve(model)

Print out model results

In [ ]:
print(results)
print("Objective function = ", model.obj())
for i in model.item:
  print(f"Number of {i} = ", model.x[i]())


Problem: 
- Name: unknown
  Lower bound: 75.0
  Upper bound: 75.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 6
  Number of binary variables: 3
  Number of integer variables: 6
  Number of nonzeros: 6
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.02
  Wallclock time: 0.02
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.040045738220214844
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective function =  75.0
Number of 0 =  0.0
Number of 1 =  0.0
Number of 2 =  25.0
